In [ ]:
import pandas as pd
import numpy as np
import glob,os

In [ ]:
os.chdir("C:/Users/Ganesh P Umarani/Desktop/Optum/Map back/7cdo")

In [ ]:
dist_file=pd.read_csv('Master_7cdo_Oncology.csv')

In [ ]:
dist_file.rename({'DISTRIBUTOR PRODUCT NUMBER':'DISTRIBUTOR PRODUCT NUMBER',
                  'DISTRIBUTOR':'DISTRIBUTOR',
                  'Manufacturer Part Number':'Manufacturer Part Number',
                  'Manufacturer':'Manufacturer',
                  'Product Description':'Product Description',
                  '11 Digit NDC Code':'11 Digit NDC Code',
                  'UNSPSC Comodity Code':'UNSPSC',
                 'Primary_Key':'Primary_Key',},axis=1,inplace=True)

In [ ]:
product_master=dist_file[['Primary_Key','DISTRIBUTOR','DISTRIBUTOR PRODUCT NUMBER','Product Description','Manufacturer Part Number','Manufacturer',
                         'UNSPSC','11 Digit NDC Code']]

In [ ]:
to_append=dist_file[['DISTRIBUTOR','DISTRIBUTOR PRODUCT NUMBER','Product Description','Manufacturer Part Number','Manufacturer',
                         'UNSPSC','11 Digit NDC Code']]

In [ ]:
f = {c: ','.join if c == 'Primary_Key' else 'first' for c in ['Primary_Key']}

product_master=product_master.groupby(['DISTRIBUTOR PRODUCT NUMBER']).agg(f).reset_index()

In [ ]:
product_master=pd.merge(product_master,to_append,on='DISTRIBUTOR PRODUCT NUMBER',how='left')

In [ ]:
product_master=product_master.groupby(['DISTRIBUTOR PRODUCT NUMBER']).first()

In [ ]:
product_master.reset_index(inplace=True)

In [ ]:
product_master.to_csv('No_dup_Master_7cdo_Oncology.csv')

In [ ]:
#append all 6 cdo's

In [ ]:
small_data=pd.read_csv('No_dup_Master_7cdo_msckesson_speciality_healthcare.csv')

In [ ]:
len(small_data)

In [ ]:
# big_data=pd.DataFrame()

In [ ]:
big_data=big_data.append(small_data,ignore_index=True)

In [ ]:
len(big_data)

In [ ]:
big_data.to_csv('No_dup_7cdo_Master.csv')

In [ ]:
os.chdir("C:/Users/Ganesh P Umarani/Desktop/Optum/Map back")

In [ ]:
done_data=pd.read_excel("Final List Distribution File.xlsx")

In [ ]:
big_data.drop(labels=['Unnamed: 0'],axis=1,inplace=True)

In [ ]:
list_of_completed_keys=done_data['Primary_Key'].tolist()

In [ ]:
output_dataframe=pd.DataFrame()

In [ ]:
for idx,row in big_data.iterrows():
    list_of_pk=str(row['Primary_Key']).split(",")
    
    if (any(x in list_of_completed_keys for x in list_of_pk)):
        row['Done']='True'
    else:
        row['Done']='False'
    
    output_dataframe=output_dataframe.append(row,ignore_index=True)
    print(len(output_dataframe))

In [ ]:
output_dataframe.columns

In [ ]:
output_dataframe_false.to_csv('Final List Distribution File matched_pk_false_v2.csv')

In [ ]:
output_dataframe_false=output_dataframe[output_dataframe.Done=='False']

In [ ]:
len(output_dataframe_false)

In [ ]:
product_master.rename({'index':'Secondary_key'},axis=1,inplace=True)

In [ ]:
product_master['Secondary_key']=product_master.index+36505

In [ ]:
product_master['Secondary_key']=product_master['Secondary_key'].astype(str)

In [ ]:
product_master['Secondary_key']='DK-'+product_master['Secondary_key']

In [ ]:
product_master.tail()

In [ ]:
product_master.to_csv('Master_Medline_Primary_Key_assigned_with_cdo_mapping_SK.csv')

In [ ]:
# Bess Mapping

In [ ]:
dist_sk=pd.read_csv('Master_Besse_Supply_distributor_file_with_cdo_mapping_HN_SK.csv')

In [ ]:
dist_sk.head()

In [ ]:
# Build look up dict

In [ ]:
look_up={}
for idx,row in dist_sk.iterrows():
    key=str(row['Secondary_key']).strip()
    values=row['Primary_Key'].split(",")
    look_up[key]=values
    

In [ ]:
def getKey(dct,value):
     return [key for key in dct.keys() if (value in dct[key])]

In [ ]:
sk_pk=dist_sk[['Secondary_key','Primary_Key']]

In [ ]:
os.chdir("C:/Users/Ganesh P Umarani/Desktop/Optum/Map back")

In [ ]:
main_file=pd.read_excel("Compile_Set2_Final_Ontario_Product_Master_v12 (1).xlsx")

In [ ]:
besse_file=main_file[main_file['Secondary_Key'].str.startswith('Besse', na = False)]

In [ ]:
besse_file_sk=pd.DataFrame()

In [ ]:
for idx,row in besse_file.iterrows():
    to_search=str(row['Secondary_Key']).strip()
    actual_sk=getKey(look_up,to_search).pop()
    row['Actual_SK']=actual_sk
    besse_file_sk=besse_file_sk.append(row,ignore_index=True)
    

In [ ]:
sk_pk_mapped_data=pd.merge(besse_file_sk,sk_pk,on='Secondary_key',how='left')

In [ ]:
final_output=pd.DataFrame()
index_with_issue=[]

In [ ]:
for j in range(len(sk_pk_mapped_data)):
    row=sk_pk_mapped_data.iloc[[j]]
    list_of_pk=str(row['Primary_key'][j]).strip().split(",")
    list_of_cols=row.columns.tolist()
    
    try:
        list_of_cols.remove('Primary_key')
    except ValueError:
        index_with_issue.append(idx)
    
    data_to_copy=row[list_of_cols]
    
    for i in range(len(list_of_pk)):
        primar_key=list_of_pk[i]
        temp_df=pd.DataFrame()
        temp_df=temp_df.append(data_to_copy,ignore_index=True)
        temp_df['Primary_Key']=primar_key
        final_output=final_output.append(temp_df,ignore_index=True)
    print(j)

In [ ]:
final_output.to_csv("Besse_mapped_unfolded_pk_v1.csv")

In [ ]:
os.chdir("C:/Users/Ganesh P Umarani/Desktop/Optum/Map back/Distributor")

In [ ]:
dist_pk=pd.read_csv('Master_Besse_Supply_distributor_file_with_cdo_mapping_HN.csv')

In [ ]:
final_output_complete=pd.merge(dist_pk,final_output,on='Primary_Key',how='left')

In [ ]:
final_output_complete.to_csv('Besse_final_output_complete.csv')